In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [12]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale) 
scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)
test_data = scaled_test_data

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

In [16]:
input_size = 784
output_size  = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
    
])

In [17]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [18]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 3s - loss: 0.3362 - accuracy: 0.9049 - val_loss: 0.1640 - val_accuracy: 0.9537
Epoch 2/5
540/540 - 3s - loss: 0.1367 - accuracy: 0.9602 - val_loss: 0.1108 - val_accuracy: 0.9677
Epoch 3/5
540/540 - 3s - loss: 0.0967 - accuracy: 0.9709 - val_loss: 0.0900 - val_accuracy: 0.9740
Epoch 4/5
540/540 - 3s - loss: 0.0755 - accuracy: 0.9773 - val_loss: 0.0836 - val_accuracy: 0.9725
Epoch 5/5
540/540 - 3s - loss: 0.0584 - accuracy: 0.9819 - val_loss: 0.0586 - val_accuracy: 0.9817


In [19]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.0750 - accuracy: 0.9765


In [20]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07. Test accuracy: 97.65%
